In [104]:
!pip install openai pandas qdrant_client sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [105]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/alfredodeza/learn-retrieval-augmented-generation/refs/heads/main/top_rated_wines.csv')
# df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
# len(data)
df.head()

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."


In [106]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [107]:
# Choose the encoder that I will use to encode my text
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [108]:
# Create my vector database (Qdrant) in memory to test
qdrant = QdrantClient(":memory:")

In [109]:
data[0]

{'name': 'CADE Howell Mountain Estate Cabernet Sauvignon 2008',
 'region': 'Howell Mountain, Napa Valley, California',
 'variety': 'Red Wine',
 'rating': 98.0,
 'notes': 'We feel that the 2008 CADE Estate Cabernet Sauvignon eclipses our blockbuster 2007, the reasons being that theweather in October 2008 was fantastic with not a drop of rain falling during ripening season and harvest.'}

In [110]:
# Create a collention
qdrant.recreate_collection(
    collection_name="wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        # Study how this Discance works
        distance=models.Distance.COSINE
    )
)

/var/folders/t1/kcf8t4w56pqgsc616g595bfh0000gn/T/ipykernel_7556/3514914447.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [111]:
qdrant.upload_points(
    collection_name="wines",
    points=[
        models.PointStruct(
            id=idx,
            # Semantic meaning to do a seach
            vector=encoder.encode(doc['notes']).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data)
    ],
)



In [116]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [117]:

# Encoder the question and add this to the vector database
hits = qdrant.search(
    collection_name="wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)

/var/folders/t1/kcf8t4w56pqgsc616g595bfh0000gn/T/ipykernel_7556/19242341.py:2: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [122]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]
for result in search_results:
    print(result)

{'name': "Greenock Creek Alice's Shiraz 2003", 'region': 'Barossa Valley, Barossa, South Australia, Australia', 'variety': 'Red Wine', 'rating': 97.0, 'notes': 'Rich and fleshy, with pretty coffee, plum, wild berry and spice notes that are smooth and polished, long and flavorful.  An extremely limited release wine from one of Australia\'s "Cult" wineries.'}
{'name': 'Cavallotto Barolo Riserva Bricco Boschis (chipped wax - 3L) 2001', 'region': 'Barolo, Piedmont, Italy', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'A wine of great structure but with elegance and complexity. Initially fruity with floral and spice aromas that open up. An excellent wine for aging.'}
{'name': 'J.J. Prum Graacher Himmelreich Gold Capsule Auslese Riesling 2003', 'region': 'Mosel, Germany', 'variety': 'White Wine', 'rating': 96.0, 'notes': '"This is really singing. Floral, apricot and mineral notes burst from the glass, and the wine is silky and impeccably balanced. Very concentrated, yet lightweight and el

In [120]:
from openai import OpenAI
client = OpenAI(
    api_key="sk-...",
    base_url="http://127.0.0.1:8080/v1",
)

completition = client.chat.completions.create(
    model="gemma3:4b",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completition.choices[0].message.content)

</font>}

Okay, fantastic! You’re looking for a sweet Italian wine. Italy has a *remarkable* range of dessert wines, and luckily for you, they're wonderfully accessible.

To narrow it down and give you the *perfect* recommendation, could you tell me a little more about what you're in the mood for?  For example:

*   **What kind of sweetness are you craving?** (e.g., Light and fruity? Rich and decadent? Something with a noticeable honey note?)
*   **What flavors do you enjoy?** (e.g., Fruit – like apricot, peach, or melon? Honey, caramel, vanilla? Nuts?  Do you like spice?)
*   **What are you pairing it with?** (e.g., Dessert, cheese, or just to enjoy on its own?)

In the meantime, here are a few initial suggestions based purely on “sweet and from Italy”:

1.  **Moscato d'Asti:** This is probably the most approachable and common sweet Italian wine. It's very light, bubbly, and bursting with intense aromas of peach, apricot, and orange blossom. It’s incredibly refreshing and often served